In [ ]:
import weave
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

!rm -rf /tmp/local-artifacts

t = pa.table({d: list(range(400)) * 1000 for d in "ABCDEFGHIJKLMNOPQRSTUVWXYZ"})
weave_t = weave.legacy.weave.ops.ArrowWeaveList(t)
weave.use(weave_t.groupby(lambda x: x['A']).map(lambda row: row.key()))

In [ ]:
import os
import cProfile
import weave
from weave.legacy.weave import storage
import random
import time
import pyarrow as pa
from PIL import Image

def create_arrow_data(n_rows):
    inner_count = int(n_rows / 25)
    rotates = []
    shears = []
    x = []
    y = []
    random.seed(0)
    for rotate in range(5):
        for shear in range(5):
            for i in range(inner_count):
                rotates.append(rotate)
                shears.append(shear)
                x.append(random.choice(['a', 'b', 'c']))
                y.append(random.randrange(10))
    table = pa.table({
        'rotate': rotates,
        'shear': shears,
        'x': x,
        'y': y,
    })
    table_list = weave.legacy.weave.ops.ArrowTableList(table)

    return storage.save(table_list)

def create_data(n_rows):
    inner_count = int(n_rows / 25)
    ims = []
    base_im = Image.linear_gradient("L")
    random.seed(0)
    for rotate in range(5):
        for shear in range(5):
            for i in range(inner_count):
                ims.append(
                    {
                        "rotate": rotate,
                        "shear": shear,
                        "y": random.choice(["a", "b", "c"]),
                        "x": random.randrange(10),
#                         "image": base_im.rotate(rotate * 4).transform(
#                             (256, 256),
#                             Image.AFFINE,
#                             (1, shear / 10, 0, 0, 1, 0),
#                             Image.BICUBIC,
#                         ),
                    }
                )

    return storage.save(ims)


def run(ref):
#     node = (
#         weave.get(ref)
#         .groupby(lambda row: weave.legacy.weave.ops.dict_(rotate=row["rotate"], shear=row["shear"]))
#         .map(lambda row: row.groupby(lambda row: row["y"]))
#         .dropna()
#         .count()
#     )
    node = (
        weave.get(ref)
        # .map(lambda row: row['rotate'] + 1)
        # .map(lambda row: row + 9)
        .groupby(lambda row: weave.legacy.weave.ops.dict_(rotate=row["rotate"], shear=row["shear"]))
        [1]
        .count()
    )
    result = weave.use(node)
    print('Run result: %s' % result)


os.system("rm -rf /tmp/local-artifacts/*")
ref = create_data(100000)
print("Data ref: ", ref)
start_time = time.time()
cProfile.run('run("%s")' % ref, "profile_stats")
print('Run time: %ss' % (time.time() - start_time))

In [ ]:
import pstats
from pstats import SortKey
p = pstats.Stats('profile_stats')
#p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_stats()
p.strip_dirs().sort_stats(SortKey.CUMULATIVE).print_callers('type_of')
#p.strip_dirs().sort_stats(SortKey.TIME).print_stats()